In [1]:
import azureml
from azureml.core import Workspace, Run
import pandas as pd
import numpy
from azureml.core.model import Model
ws = Workspace.from_config()

Found the config file in: C:\Users\zhpek\Desktop\C4I\Challenge 4\aml_config\config.json


In [2]:
import logging
import os
import random
import re

from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd
from sklearn import datasets

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.run import Run
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun

In [3]:
ws.experiments

{'train-on-remote-vm': Experiment(Name: train-on-remote-vm,
 Workspace: pekamlworkspace),
 'aks-compute-trial': Experiment(Name: aks-compute-trial,
 Workspace: pekamlworkspace),
 'sklearn-mnist': Experiment(Name: sklearn-mnist,
 Workspace: pekamlworkspace),
 'automl-local-classification': Experiment(Name: automl-local-classification,
 Workspace: pekamlworkspace),
 'tf-mnist': Experiment(Name: tf-mnist,
 Workspace: pekamlworkspace),
 'automl-pipeline-classification': Experiment(Name: automl-pipeline-classification,
 Workspace: pekamlworkspace),
 '10k-Diabetes-with-AutoML': Experiment(Name: 10k-Diabetes-with-AutoML,
 Workspace: pekamlworkspace),
 'c4ts-challenge4': Experiment(Name: c4ts-challenge4,
 Workspace: pekamlworkspace)}

In [4]:
experiment_name = 'c4ts-challenge4' # Replace this with any project name from previous cell
experiment = ws.experiments[experiment_name]
#ml_run = AutoMLRun(experiment=experiment, run_id=run_id)
selected_metric = 'AUC_weighted'
#summary_df = pd.DataFrame(index = ['Type', 'Status', 'Primary Metric', 'Iterations', 'Compute', 'Name'])
#pattern = re.compile('^AutoML_[^_]*$')
runs = list(experiment.get_runs())
model_dict = {}
for run in runs[:2]:
    aml_run = AutoMLRun(experiment = experiment, run_id = run.id)
    print(run.id)
    best_run, model = aml_run.get_output()
    model_dict[best_run.id]= {'model': model, 'run': best_run}
    metrics = best_run.get_metrics()
    if selected_metric in metrics:
        model_dict[best_run.id][selected_metric] = metrics[selected_metric]

AutoML_ece6d8e3-f37a-49c3-b601-c35261eab752
AutoML_de68d57f-dbc9-445a-bd87-229040fbabe0


In [5]:
acc = [model_dict[k][selected_metric] for k in model_dict]
runlist = [model_dict[k]['run'] for k in model_dict]
best_run = runlist[acc.index(max(acc))]
print(best_run)

Run(Experiment: c4ts-challenge4,
Id: AutoML_de68d57f-dbc9-445a-bd87-229040fbabe0_2,
Type: None,
Status: Completed)


In [6]:
models_df = pd.DataFrame(model_dict).T
models_df

,AUC_weighted,model,run
AutoML_ece6d8e3-f37a-49c3-b601-c35261eab752_2,0.787947,"Pipeline(memory=None,\n steps=[('prefitted...","Run(Experiment: c4ts-challenge4,\nId: AutoML_e..."
AutoML_de68d57f-dbc9-445a-bd87-229040fbabe0_2,0.788992,"Pipeline(memory=None,\n steps=[('prefitted...","Run(Experiment: c4ts-challenge4,\nId: AutoML_d..."


In [7]:
run_id = models_df[selected_metric].astype(float).idxmax()
best_run = models_df.loc[run_id, 'run']
best_model = models_df.loc[run_id, 'model']
#best_run = df['model',run_id]

In [38]:
model = best_run.register_model(model_name = 'TestModelAutoML')
model_id = model.id
model_id

'TestModelAutoML:12'

In [39]:
run_id[:-2]

'AutoML_de68d57f-dbc9-445a-bd87-229040fbabe0'

In [40]:
rundl = AutoMLRun(experiment = experiment, run_id = run_id[:-2])

In [41]:
import pickle

files = rundl.get_file_names()

for f in files:
    rundl.download_file(f)

In [42]:
#%%writefile score_sparkml.py
score_script = """
 
import pickle
import json
import numpy
from sklearn.externals import joblib
from azureml.core.model import Model
import pandas as pd
import numpy as np
from utils import scaler, cleanLongLat

pca_transform = pickle.load(open('pca_transform.pkl','rb'))
scaler = pickle.load(open('scaler.pkl','rb'))
        
def init():
    global model
    global pca_transform
    global scaler
    model_id = {model_id}
    model_path = Model.get_model_path(model_name = model_id)
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)
    pca_transform = pickle.load(open('pca_transform.pkl', 'rb'))
    scaler = pickle.load(open('scaler.pkl','rb'))

def run(rawdata):
    try:
        df = json.loads(j_test)['data']
        df = pd.DataFrame.from_records(df)
        df['Latitude'] = cleanLongLat(df['Latitude'])
        df['Longitude'] = cleanLongLat(df['Longitude'])
        df.drop(['Machine_ID', 'District'], axis=1, inplace=True)
        df.drop('Failure_NextHour', axis = 1, inplace=True)
        df = df.apply(pd.to_numeric)
        data = pca_transform.transform(scaler.scale(df))
        result = best_model.predict(data) #Changed to best_model here
    except Exception as e:
        result = str(e)
        return json.dumps({{"error": result}})
    return json.dumps({{"result":result.tolist()}}) 
    
""".format(model_id = model_id)

with open("score.py", "w") as file:
    file.write(score_script)

In [43]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")
myenv.add_conda_package("pandas")
myenv.add_conda_package("numpy")
myenv.add_pip_package("pynacl==1.2.1")

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [44]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "c4ts4",  "method" : "automl"}, 
                                               description='C4TS Challenge 4')

In [45]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage

# configure the image
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="myenv.yml")

service = Webservice.deploy_from_model(workspace=ws,
                                       name='c4ts-svc',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=image_config)

service.wait_for_deployment(show_output=True)

Creating image
Image creation operation finished for image c4ts-svc:1, operation "Succeeded"
Creating service


WebserviceException: Received bad response from Model Management Service:
Response Code: 500
Headers: {'Date': 'Mon, 07 Jan 2019 09:30:53 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'api-supported-versions': '2018-03-01-preview, 2018-11-19', 'x-ms-client-request-id': 'bd2b22ce8e6343bcba6d0a6a5ce7225b', 'x-ms-client-session-id': '', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload'}
Content: b'{"code":"InternalServerError","statusCode":500,"message":"An internal server error occurred. Please try again. If the problem persists, contact support"}'

In [46]:
print(service.scoring_uri)

NameError: name 'service' is not defined

In [16]:
import csv
import json
with open('AssetData_Historical.csv') as f:
    reader = csv.DictReader(f)
    titles = reader.fieldnames
    j_test = json.dumps({'data':[row for row in reader]})
    
result = service.run(input_data = j_test)

In [36]:
import csv
import json
with open('AssetData_Historical.csv') as f:
    reader = csv.DictReader(f)
    titles = reader.fieldnames
    j_test = json.dumps({'data':[row for row in reader]})
    
def run(rawdata):
    try:
        df = json.loads(j_test)['data']
        df = pd.DataFrame.from_records(df)
        df['Latitude'] = cleanLongLat(df['Latitude'])
        df['Longitude'] = cleanLongLat(df['Longitude'])
        df.drop(['Machine_ID', 'District'], axis=1, inplace=True)
        df.drop('Failure_NextHour', axis = 1, inplace=True)
        df = df.apply(pd.to_numeric)
        data = pca_transform.transform(scaler.scale(df))
        result = best_model.predict(data) #Changed to best_model here
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})
    return json.dumps({"result":result.tolist()}) #removed double brackets here

run(j_test)

C:\Anaconda\envs\AzureML\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


'{"result": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [19]:
df = json.loads(j_test)['data']
df = pd.DataFrame.from_records(df)
df['Latitude'] = cleanLongLat(df['Latitude'])
df['Longitude'] = cleanLongLat(df['Longitude'])
df.drop(['Machine_ID', 'District'], axis=1, inplace=True)
df.drop('Failure_NextHour', axis = 1, inplace=True)
data = pca_transform.transform(scaler.scale(df))
result = best_model.predict(data) #Changed to best_model here

df = json.loads(j_test)['data']
df = pd.DataFrame.from_records(df)
df['Latitude'] = cleanLongLat(df['Latitude'])
df['Longitude'] = cleanLongLat(df['Longitude'])
df.drop(['Machine_ID', 'District'], axis=1, inplace=True)
df.drop('Failure_NextHour', axis = 1, inplace=True)
df = df.apply(pd.to_numeric)
data = pca_transform.transform(scaler.scale(df))

df.apply(pd.to_numeric)
#data = pca_transform.transform(scaler.scale(df))

#pd.DataFrame.from_records(data['messages']).map.apply(pd.Series)
pd.DataFrame.from_records(j_load)

from utils import scaler, cleanLongLat

pca_transform = pickle.load(open('pca_transform.pkl','rb'))
scaler = pickle.load(open('scaler.pkl','rb'))


#s = scaler_obj.scale(X)
#p = pca_transform.transform(s)
#j = json.dumps({'data':p.tolist()})

#df = json.loads(j_test)['data']
#df = pd.DataFrame.from_records(df)
#df['Latitude'] = cleanLongLat(df['Latitude'])
#df['Longitude'] = cleanLongLat(df['Longitude'])
#df.drop(['Machine_ID', 'District'], axis=1, inplace=True)
#df.drop('Failure_NextHour', axis = 1, inplace=True)
#df = df.apply(pd.to_numeric)
#data = pca_transform.transform(scaler.scale(df))